# Optimize a Neural Network with a genetic algorithm



In [20]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class GA_NeuralNet:
    def __init__(self, pop_size=10, generations=10, mutation_rate=0.01, crossover_rate=0.7):
        self.pop_size = pop_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.population = self.initialize_population()

    def initialize_population(self):
        population = []
        for _ in range(self.pop_size):
            individual = {
                'hidden_layer_sizes': np.random.randint(10, 100),
                'learning_rate_init': np.random.uniform(0.0001, 0.1),
                'max_iter': np.random.randint(100, 1000)
            }
            population.append(individual)
        return population

    def fitness(self, individual, X_train, y_train, X_val, y_val):
        model = MLPClassifier(hidden_layer_sizes=(individual['hidden_layer_sizes'],),
                              learning_rate_init=individual['learning_rate_init'],
                              max_iter=individual['max_iter'])
        model.fit(X_train, y_train)
        predictions = model.predict(X_val)
        accuracy = accuracy_score(y_val, predictions)
        return accuracy

    def selection(self, fitnesses):
        probabilities = fitnesses / np.sum(fitnesses)
        selected_indices = np.random.choice(len(self.population), size=len(self.population), p=probabilities)
        return [self.population[i] for i in selected_indices]

    def crossover(self, parent1, parent2):
        if np.random.rand() < self.crossover_rate:
            child1 = {
                'hidden_layer_sizes': parent1['hidden_layer_sizes'],
                'learning_rate_init': parent2['learning_rate_init'],
                'max_iter': parent1['max_iter']
            }
            child2 = {
                'hidden_layer_sizes': parent2['hidden_layer_sizes'],
                'learning_rate_init': parent1['learning_rate_init'],
                'max_iter': parent2['max_iter']
            }
            return child1, child2
        else:
            return parent1, parent2

    def mutate(self, individual):
        if np.random.rand() < self.mutation_rate:
            individual['hidden_layer_sizes'] = np.random.randint(10, 100)
        if np.random.rand() < self.mutation_rate:
            individual['learning_rate_init'] = np.random.uniform(0.0001, 0.1)
        if np.random.rand() < self.mutation_rate:
            individual['max_iter'] = np.random.randint(100, 1000)
        return individual

    def run(self, X_train, y_train, X_val, y_val):
        best_solution = None
        best_fitness = 0

        for generation in range(self.generations):
            fitnesses = np.array([self.fitness(ind, X_train, y_train, X_val, y_val) for ind in self.population])
            new_population = self.selection(fitnesses)

            for i in range(0, len(self.population), 2):
                parent1, parent2 = new_population[i], new_population[i+1]
                child1, child2 = self.crossover(parent1, parent2)
                new_population[i], new_population[i+1] = self.mutate(child1), self.mutate(child2)

            self.population = new_population
            current_best = max(self.population, key=lambda ind: self.fitness(ind, X_train, y_train, X_val, y_val))
            current_fitness = self.fitness(current_best, X_train, y_train, X_val, y_val)

            if current_fitness > best_fitness:
                best_solution, best_fitness = current_best, current_fitness

            # print(f"Generation {generation+1}: Best Fitness = {best_fitness}")

        return best_solution, best_fitness

## Test

In [32]:
import warnings
warnings.filterwarnings('ignore')

data = load_iris()
X_train, X_val, y_train, y_val = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

def run_ga_neural_net(X_train, y_train, X_val, y_val):
    ga_nn = GA_NeuralNet()
    best_solution, best_fitness = ga_nn.run(X_train, y_train, X_val, y_val)
    return best_solution, best_fitness

num_runs = 10

for i in range(num_runs):
    best_solution, best_fitness = run_ga_neural_net(X_train, y_train, X_val, y_val)
    print(f"Run {i+1}: Best Solution = {best_solution}")

Run 1: Best Solution = {'hidden_layer_sizes': 85, 'learning_rate_init': 0.040590198398753385, 'max_iter': 612}
Run 2: Best Solution = {'hidden_layer_sizes': 95, 'learning_rate_init': 0.013832377489114275, 'max_iter': 416}
Run 3: Best Solution = {'hidden_layer_sizes': 89, 'learning_rate_init': 0.008443958286750054, 'max_iter': 903}
Run 4: Best Solution = {'hidden_layer_sizes': 30, 'learning_rate_init': 0.02168128343341507, 'max_iter': 914}
Run 5: Best Solution = {'hidden_layer_sizes': 57, 'learning_rate_init': 0.040824153167274835, 'max_iter': 635}
Run 6: Best Solution = {'hidden_layer_sizes': 72, 'learning_rate_init': 0.05682821649721228, 'max_iter': 442}
Run 7: Best Solution = {'hidden_layer_sizes': 26, 'learning_rate_init': 0.09001325596072808, 'max_iter': 809}
Run 8: Best Solution = {'hidden_layer_sizes': 86, 'learning_rate_init': 0.06660495146313739, 'max_iter': 945}
Run 9: Best Solution = {'hidden_layer_sizes': 64, 'learning_rate_init': 0.025022689300375198, 'max_iter': 222}
Run 1